In [2]:
import pickle
import numpy as np
X = pickle.load(open('Pickles/Features.pkl','rb'))
Y = pickle.load(open('Pickles/Labels.pkl','rb'))

x = np.reshape(X[0], (1, 601, 1))
print(x.shape)

(1, 601, 1)


In [3]:
from tensorflow.keras import  models
model = models.load_model('cnn_mitbih.h5')
model.summary()
yP = model.predict(x)
print(yP)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 601, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 597, 16)           96        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 593, 16)           1296      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 296, 16)           0         
_________________________________________________________________
dropout (Dropout)            (None, 296, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 294, 32)           1568      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 292, 32)           3104  

In [4]:
# Convert the model to the TensorFlow Lite format without quantization
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("ECG.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("ECG.tflite")
print("Model is %d bytes" % basic_model_size)

INFO:tensorflow:Assets written to: C:\Users\SOUMYA\AppData\Local\Temp\tmp_3rzs7ll\assets
Model is 1030940 bytes


In [2]:
!echo "const unsigned char model[] = {" > model.h
!cat ECG.tflite | xxd -i      >> model.h
!echo "};"                              >> model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")


'xxd' is not recognized as an internal or external command,
operable program or batch file.


Header file, model.h, is 72 bytes.


In [5]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="ECG.tflite")
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

# input details
print(input_details)

# output details
print(output_details)

[{'name': 'input_1', 'index': 0, 'shape': array([  1, 601,   1]), 'shape_signature': array([ -1, 601,   1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 74, 'shape': array([1, 2]), 'shape_signature': array([-1,  2]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [12]:
# input_details[0]['index'] = the index which accepts the input
input_data = np.array(x, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

# run the inference
interpreter.invoke()

# output_details[0]['index'] = the index which provides the input
output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)

[[0.08258546 0.9174145 ]]


In [11]:
input_data.shape

(1, 601, 1)